In [ ]:
"""
p-Laplacian regression (GraphLearning) — basic experiment runner

Installs (recommended):
    pip install numpy pandas scikit-learn graphlearning

Optional (only if you later add plots):
    pip install matplotlib

Notes:
- This script evaluates p-Laplacian regression using a *20% labeled / 80% unlabeled* split.
- We use KFold to generate 5 disjoint 20% folds; each fold is treated as the labeled set.
- Features are standardized using ONLY the labeled set (to avoid leakage).
"""

from __future__ import annotations

import numpy as np
import pandas as pd
import graphlearning as gl

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


# ----------------------------
# User configuration
# ----------------------------
FILE_PATHS = {
    "Male Data": "/Users/gyaneshwaragrahari/Documents/GitHub/Pennington_Project/Data/male.csv",
    "Female Data": "/Users/gyaneshwaragrahari/Documents/GitHub/Pennington_Project/Data/female.csv",
    "Complete Data": "/Users/gyaneshwaragrahari/Documents/GitHub/Pennington_Project/Data/complete_data.csv",
}

EXCLUDED_COLUMNS = ["0", "PPT ID", "Site", "Gender", "BMD - Total", "ALM", "% fat - Total", "Race"]
TARGET_COLUMNS = ["ALM", "% fat - Total", "BMD - Total"]

K_VALUES = [10, 30, 50]
P_VALUES = [2, 5, 10, 100, 500, 1000, 5000, 10000, 50000, 100000]

N_SPLITS = 5   # KFold: each fold is 20% of the data
N_RUNS = 10    # independent shuffles (random_state = run)


# ----------------------------
# Helpers
# ----------------------------
def load_xy(csv_path: str, excluded_columns: list[str], target_column: str) -> tuple[np.ndarray, np.ndarray]:
    """Load CSV, return (X, y)."""
    df = pd.read_csv(csv_path)

    missing = [c for c in excluded_columns + [target_column] if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in {csv_path}: {missing}")

    X = df.drop(columns=excluded_columns).to_numpy(dtype=float)
    y = df[target_column].to_numpy(dtype=float)
    return X, y


def relative_rmse(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Relative RMSE (%).
    Uses RMS(y_true) in the denominator (common for normalized RMSE).
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    denom = np.sqrt(np.mean(y_true**2)) + 1e-12  # avoid division by 0
    return 100.0 * rmse / denom


# ----------------------------
# Main experiment
# ----------------------------
def main() -> dict:
    # rmse_values[target][dataset] = array shape (len(K_VALUES), len(P_VALUES))
    rmse_values: dict[str, dict[str, np.ndarray]] = {
        target: {ds: np.zeros((len(K_VALUES), len(P_VALUES)), dtype=float) for ds in FILE_PATHS}
        for target in TARGET_COLUMNS
    }

    scaler = StandardScaler()

    for target in TARGET_COLUMNS:
        for dataset_name, csv_path in FILE_PATHS.items():
            X, y = load_xy(csv_path, EXCLUDED_COLUMNS, target)

            for run in range(N_RUNS):
                kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=run)

                for labeled_idx, unlabeled_idx in kf.split(X):
                    # By convention, we treat the 20% fold as the *labeled* set
                    X_labeled, y_labeled = X[labeled_idx], y[labeled_idx]
                    X_unlabeled, y_unlabeled = X[unlabeled_idx], y[unlabeled_idx]

                    # Standardize using ONLY labeled data (prevents leakage)
                    X_labeled = scaler.fit_transform(X_labeled)
                    X_unlabeled = scaler.transform(X_unlabeled)

                    # Combine for graph construction (GraphLearning expects full dataset)
                    X_all = np.vstack([X_labeled, X_unlabeled])
                    y_all = np.concatenate([y_labeled, y_unlabeled])

                    labeled_nodes = np.arange(len(y_labeled))  # labeled nodes come first
                    unlabeled_mask = np.zeros(len(y_all), dtype=bool)
                    unlabeled_mask[len(y_labeled):] = True

                    for i, k in enumerate(K_VALUES):
                        W = gl.weightmatrix.knn(X_all, k)
                        G = gl.graph(W)

                        for j, p in enumerate(P_VALUES):
                            # p-Laplace regression: returns predictions on ALL nodes
                            y_hat = G.plaplace(labeled_nodes, y_labeled, p)

                            score = relative_rmse(y_true=y_all[unlabeled_mask],
                                                  y_pred=y_hat[unlabeled_mask])
                            rmse_values[target][dataset_name][i, j] += score

            # Average over folds * runs
            rmse_values[target][dataset_name] /= (N_SPLITS * N_RUNS)

    return rmse_values


if __name__ == "__main__":
    results = main()

    # Minimal printing; you can replace this with saving to CSV/LaTeX.
    for target, ds_map in results.items():
        print(f"\n=== Target: {target} ===")
        for ds_name, arr in ds_map.items():
            print(f"\n{ds_name} (rows=k, cols=p):")
            print(arr)
